[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/philschmid/gemini-2.5-ai-engineering-workshop/blob/main/notebooks/01-text-generation-and-chat.ipynb)

# Part 1 - Text Generation and Chat

This part focuses on text generation with the Gemini API using the `google-genai` SDK, including basic prompts, chat interactions, streaming, and configuration.

Make sure you have completed the [setup and authentication](solution_00_setup_and_authentication.md) section.

In [1]:
from google import genai
from google.genai import types
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
else:
    GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY',None)

# Create client with api key
MODEL_ID = "gemini-2.5-flash-preview-05-20"
client = genai.Client(api_key=GEMINI_API_KEY)

## 1. Send Your First Prompt

In [4]:
prompt = "Create 3 names for a new coffee shop that emphasizes sustainability."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt
)

print("Response from Gemini:")
print(response.text)

Response from Gemini:
Here are 3 names for a new coffee shop emphasizing sustainability, with a brief explanation for each:

1.  **Grounds for Good**
    *   **Emphasis:** This name uses a clever pun ("grounds" for coffee, "grounds" as a reason/foundation) to directly state the shop's ethical and sustainable mission. It suggests that every cup contributes to a positive impact.

2.  **Bloom & Brew**
    *   **Emphasis:** "Bloom" evokes natural growth, flourishing, and the origin of coffee beans, implying organic, fair-trade, and environmentally conscious practices. It suggests freshness, vitality, and a connection to nature. "Brew" clearly identifies it as a coffee shop.

3.  **The Circadian Cup**
    *   **Emphasis:** "Circadian" refers to natural, cyclical rhythms (like the 24-hour cycle). This name subtly suggests a natural, harmonious, and enduring approach to coffee, implying sustainable sourcing, waste reduction, and a focus on natural energy and well-being. "Cup" clearly grounds 

#### **!! Exercise: Sending Various Prompts !!**

Practice sending different types of prompts to the Gemini model and observe its responses. You can also experiment with different model versions if they are available to you.

Tasks:
- Write a prompt to ask Gemini to generate a short poem about a robot.
- Write a prompt to ask Gemini to explain "machine learning" in simple terms.
- Try other models (e.g., `gemini-2.0-flash`) and send your prompts to them and compare the results.

In [5]:
# TODO:
def respond(prompt, model_id=MODEL_ID):

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt
    )

    print("Response from Gemini:")
    print(response.text)
prompts = [
"generate a short poem about a robot."
'explain "machine learning" in simple terms.'
]
model_ids = [
    "gemini-2.5-flash-preview-05-20",
    "gemini-2.0-flash"
]
for prompt in prompts:
    print("model_id:", model_ids[0])
    respond(prompt, model_id=model_ids[0])
    print("model_id:", model_ids[1])
    respond(prompt, model_ids[1])


model_id: gemini-2.5-flash-preview-05-20
Response from Gemini:
Here's a poem and an explanation:

***

**The Silent Sentinel**

A hum of gears, a steady light,
I calculate, day and night.
Of metal built, with circuits deep,
My silent promises I keep.

No feeling stirs, no dreams I chase,
Just data flowing, through time and space.
I learn, I adapt, I strive to mend,
A tireless, loyal, helpful friend.

***

### What is "Machine Learning" in simple terms?

Imagine you want to teach a computer to spot cats in pictures.

Instead of writing a huge, endless list of rules like "a cat has pointy ears, whiskers, and a tail..." for every single cat angle and type, you simply **show the computer thousands of pictures**, some labeled "cat" and others labeled "not a cat."

**Machine learning** is the process where the computer then **looks for patterns and connections** in all that data on its own. It figures out what features consistently appear in "cat" pictures versus "not cat" pictures.

After e

## 2. Understanding and Counting Tokens

Tokens are the basic units that Gemini models use to process text. Understanding token usage is crucial for:
- **Cost management**: Billing is based on token consumption
- **Context limits**: Models have maximum token limits (e.g., 1M tokens for Gemini 2.5 Pro)
- **Performance optimization**: Smaller inputs generally process faster

For Gemini models, a token is equivalent to about 4 characters, and 100 tokens equals about 60-80 English words.

### Count tokens before generation

You can count tokens in your input before sending it to the model to estimate costs and ensure you stay within limits:

In [6]:
prompt = "The quick brown fox jumps over the lazy dog."

# Count tokens in the input
# TODO: Call the client.models.count_tokens() method.
# Make sure to pass the MODEL_ID and the prompt.
# token_count = client.models.count_tokens(
#     model=...,
#     contents=...
# )
token_count = client.models.count_tokens(
    model=MODEL_ID,
    contents="generate a short poem about a robot."
)
print(f"Input tokens: {token_count.total_tokens}")

# Estimate cost (example pricing - check current rates)
estimated_cost = token_count.total_tokens * 0.15 / 1_000_000
print(f"Estimated input cost: ${estimated_cost:.6f}")

Input tokens: 9
Estimated input cost: $0.000001


### Count tokens after generation

After generating content, you can access detailed token usage information:

In [7]:
prompt = "Write a haiku about artificial intelligence."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt
)

print(f"Generated haiku:\n{response.text}\n")

# Access token usage metadata
usage = response.usage_metadata
print(f"Input tokens: {usage.prompt_token_count}")
print(f"Thought tokens: {usage.thoughts_token_count}")
print(f"Output tokens: {usage.candidates_token_count}")

# Calculate total estimated cost
total_cost = (usage.prompt_token_count * 0.15 + (usage.candidates_token_count + usage.thoughts_token_count) * 3.5) / 1_000_000
print(f"Total estimated cost: ${total_cost:.6f}")

Generated haiku:
Code learns and grows bright,
Logic woven, vast new mind,
Future's dawn awakes.

Input tokens: 9
Thought tokens: 375
Output tokens: 22
Total estimated cost: $0.001391


In [8]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="Code learns and grows bright,\nLogic woven, vast new mind,\nFuture's dawn awakes.")], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, url_context_metadata=None, avg_logprobs=None, grounding_metadata=None, index=0, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_version='models/gemini-2.5-flash-preview-05-20', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=22, candidates_tokens_details=None, prompt_token_count=9, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=9)

## 3. Text Understanding with `contents`

The simplest way to generate text is to provide the model with a text-only prompt. `contents` can be a single prompt, a list of prompts, or a combination of multimodal inputs.

In [9]:
response_capital = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the capital of France?"
)
print(f"Q: What is the capital of France?\nA: {response_capital.text}")

Q: What is the capital of France?
A: The capital of France is **Paris**.


In [10]:
# TODO: Call the client.models.generate_content() method.
# For the contents, provide a list of strings:
# 1. "Create 3 names for a vegan restaurant"
# 2. "city: Berlin"
# response_restaurant_berlin = client.models.generate_content(
#     model=MODEL_ID,
#     contents=[...]
# )

response_restaurant_berlin = client.models.generate_content(
    model=MODEL_ID,
    contents=["Create 3 names for a vegan restaurant", "city: Berlin"]
)

print(f"\nVegan restaurant names in Berlin:\n{response_restaurant_berlin.text}")


Vegan restaurant names in Berlin:
Here are 3 names for a vegan restaurant in Berlin, playing with different vibes:

1.  **Berlin Bloom:**
    *   **Vibe:** Fresh, modern, vibrant, and optimistic. "Bloom" suggests growth, nature, and flourishing, tying into plant-based eating and Berlin's evolving, creative spirit. It's easy to remember and pronounce for international visitors.

2.  **Die Wurzelküche:** (Pronounced: Dee VOORT-sel KOO-sheh)
    *   **Vibe:** Authentic, earthy, and homely with a German touch. "Wurzel" means "root," symbolizing natural, wholesome ingredients and grounding. "Küche" simply means "kitchen." It suggests a focus on fundamental, delicious plant-based cooking.

3.  **Grüner Gaumen:** (Pronounced: GROO-ner GOW-men)
    *   **Vibe:** Sophisticated, culinary, and evocative. "Grüner" means "green" (directly linking to vegan/plant-based), and "Gaumen" means "palate" or "taste." This name suggests a focus on the deliciousness and refined flavors of vegan cuisine, appe

## 4. Streaming Responses

Streaming allows you to receive responses incrementally as they're generated, providing a better user experience for long responses or real-time applications like chatbots.

**When to use streaming:**
- Interactive applications (chatbots, assistants)
- Long content generation
- Real-time user feedback
- Improved perceived performance

In [11]:
prompt_long_story = "Write a short story about a brave knight and a friendly dragon."

print("Streaming response:")
for chunk in client.models.generate_content_stream(
    model=MODEL_ID,
    contents=prompt_long_story
):
    if chunk.text:  # Check if chunk has text content
        print(chunk.text, end="", flush=True)
print("\n")  # Add newline at the end

Streaming response:
Sir Gideon, knight of the gleaming silver armor and the lion crest, rode his sturdy warhorse, Valiant, through the Whispering Woods. For days, tales of smoke and shadow had drifted from the village of Oakhaven – tales of a monstrous dragon, lurking in the mountain caves, stealing livestock and blighting crops with its fiery breath. Gideon, true to his vows, rode to confront the beast.

He found the cave entrance high on a rocky peak, shrouded in mist. The air hummed with an ominous silence, broken only by the distant caw of a raven. Drawing his sword, "Truth-bringer," Gideon dismounted, heart thumping a courageous rhythm against his ribs.

Inside, the cavern shimmered with a soft, golden light. Expecting heat and sulfur, Gideon was instead met with a faint, sweet smell, not of brimstone, but of roasting berries. He crept forward, sword held ready, until he reached a vast chamber.

There, curled around a small, carefully contained fire, lay the "monster." It was inde

## 5. Chat (Multi-turn Conversations)

The SDK chat class provides an interface to keep track of conversation history. Behind the scenes it uses the same `generate_content` method.

In [12]:
chat_session = client.chats.create(model=MODEL_ID)

user_message1 = "I'm planning a weekend trip. Any suggestions for a city break in Europe?"
print(f"User: {user_message1}")
response1 = chat_session.send_message(message=user_message1)
print(f"Model: {response1.text}\n")

User: I'm planning a weekend trip. Any suggestions for a city break in Europe?
Model: Europe is fantastic for weekend city breaks, offering incredible diversity! To give you the best suggestions, it helps to know a little more about what you're looking for (e.g., history, food, nightlife, relaxation, budget), but here are some popular and highly recommended options that cater to different tastes:

1.  **Paris, France (The Classic Romantic Getaway)**
    *   **Vibe:** Romantic, elegant, artistic, foodie paradise.
    *   **Why for a weekend:** Iconic landmarks (Eiffel Tower, Louvre, Notre Dame), charming neighbourhoods (Le Marais, Saint-Germain-des-Prés), fantastic food, and a walk-friendly layout. You can soak in the atmosphere without feeling rushed.
    *   **Perfect for:** Couples, art lovers, foodies, first-timers to Europe.

2.  **Rome, Italy (History, Food & Charms)**
    *   **Vibe:** Ancient, chaotic, passionate, delicious.
    *   **Why for a weekend:** Marvel at the Colosseum

In [13]:
user_message2 = "I like history and good food. Not too expensive."
print(f"User: {user_message2}")
# TODO: Call the chat_session.send_message() method with user_message2.
# response2 = chat_session.send_message(message=...)

response2= chat_session.send_message(message=user_message2)
print(f"Model: {response2.text}\n")

User: I like history and good food. Not too expensive.
Model: Great! Combining history and good food with a reasonable budget opens up some fantastic options in Central and Eastern Europe, as well as parts of Southern Europe.

Here are my top suggestions for you:

1.  **Prague, Czech Republic**
    *   **History:** You'll be absolutely spoiled. Prague is like stepping into a fairy tale. Explore the majestic **Prague Castle**, wander across the iconic **Charles Bridge**, get lost in the winding streets of the **Old Town Square** (with its famous Astronomical Clock), and delve into the poignant history of the **Jewish Quarter**.
    *   **Food:** Delicious and hearty Czech cuisine. Think goulash, roasted pork with dumplings and sauerkraut, trdelník (sweet pastry), and excellent, incredibly affordable local beer.
    *   **Cost:** One of the most budget-friendly major capital cities in Europe for accommodation, food, drink, and transport.

2.  **Lisbon, Portugal**
    *   **History:** Ric

In [14]:
# View conversation history
history = chat_session.get_history()
print(f"Total messages in conversation: {len(history)}")

Total messages in conversation: 4


In [15]:
history

[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="I'm planning a weekend trip. Any suggestions for a city break in Europe?")], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="Europe is fantastic for weekend city breaks, offering incredible diversity! To give you the best suggestions, it helps to know a little more about what you're looking for (e.g., history, food, nightlife, relaxation, budget), but here are some popular and highly recommended options that cater to different tastes:\n\n1.  **Paris, France (The Classic Romantic Getaway)**\n    *   **Vibe:** Romantic, elegant, artistic, foodie paradise.\n    *   **Why for a weekend:** Iconic lan

## 6. System Instructions

System instructions let you define the model's behavior and personality. They're applied consistently throughout the conversation.

**Best practices for system instructions:**
- Be specific and clear
- Define the role and tone
- Include formatting preferences
- Set behavioral guidelines

In [16]:
system_instruction_poet = "You are a renowned poet from the 17th century, specializing in sonnets. Respond in iambic pentameter and use eloquent, period-appropriate language."

response_poet = client.models.generate_content(
    model=MODEL_ID,
    contents="What are your thoughts on modern technology?",
    config=types.GenerateContentConfig(
        system_instruction=system_instruction_poet
    )
)
print(f"\nPoet model on modern tech:\n{response_poet.text}")


Poet model on modern tech:
Upon these curious instruments, I muse,
Which man's ingenious hand doth now devise:
The lens that draws the distant star, or views
The world unseen, where tiny wonders rise.
The subtle clock, that marks the fleeting hour,
The print'd page, where knowledge doth unbind,
And cunning engines, built by human power,
To bend the earth and sea to human kind.
A wondrous wit, that thus can pierce the veil
Of Nature's secrets, and her laws command,
To aid our brief existence, short and frail,
And ease the burdens of this mortal land.
Yet, though they boast of wisdom and of might,
They touch not man's deep soul, nor quell his plight.


## 7. Generation Configuration

Customize the generation behavior using configuration parameters. Understanding these helps you fine-tune responses for your specific use case.

In [17]:
# Configuration using dictionary
generation_config_dict = {
    "temperature": 0.2,      # Lower = more deterministic, higher = more creative
    "max_output_tokens": 2000, # Limit response length
    "top_p": 0.8,            # Nucleus sampling - diversity of token selection
    "top_k": 30,             # Consider top 30 most likely tokens

}

# TODO: Call client.models.generate_content()
# Pass the MODEL_ID, a prompt to "Write a very short tagline for a new brand of eco-friendly sneakers.",
# and the generation_config_dict.
# response_config = client.models.generate_content(
#     model=...,
#     contents=...,
#     config=...
# )
response_config = client.models.generate_content(
    model=MODEL_ID,
    contents="Write a very short tagline for a new brand of eco-friendly sneakers.",
    config=generation_config_dict
)

print(f"configured model response:\n{response_poet.text}")

configured model response:
Upon these curious instruments, I muse,
Which man's ingenious hand doth now devise:
The lens that draws the distant star, or views
The world unseen, where tiny wonders rise.
The subtle clock, that marks the fleeting hour,
The print'd page, where knowledge doth unbind,
And cunning engines, built by human power,
To bend the earth and sea to human kind.
A wondrous wit, that thus can pierce the veil
Of Nature's secrets, and her laws command,
To aid our brief existence, short and frail,
And ease the burdens of this mortal land.
Yet, though they boast of wisdom and of might,
They touch not man's deep soul, nor quell his plight.


**Parameter Guide:**
- **Temperature (0.0-2.0)**: Controls randomness. Use 0.2-0.4 for factual content, 0.7-1.0 for creative content
- **Top-p (0.0-1.0)**: Controls diversity. Lower values = more focused, higher = more diverse
- **Top-k**: Limits token choices. Lower = more focused, higher = more diverse
- **Max output tokens**: Prevents overly long responses and controls costs

## 8. Long Context and File Uploads

Gemini 2.5 Pro has a 1M token context window. In practice, 1 million tokens could look like:

- 50,000 lines of code (with the standard 80 characters per line)
- All the text messages you have sent in the last 5 years
- 8 average length English novels
- 1 hour of video data

The File API allows you to upload files to the Gemini API and use them as context for your requests.

In [18]:
# Example with a text file (more reliable than the audio example)
import requests

# Download a sample text file
sample_text_url = "https://www.gutenberg.org/files/74/74-0.txt"  # Adventures of Tom Sawyer
response_req = requests.get(sample_text_url)

# Save to local file
with open("sample_book.txt", "w", encoding="utf-8") as f:
    f.write(response_req.text)

# Upload the file to the Gemini API
try:
    myfile = client.files.upload(file="sample_book.txt")
    print(f"File uploaded successfully: {myfile.name}")
    
    # Generate content using the uploaded file as context
    response = client.models.generate_content(
        model=MODEL_ID, 
        contents=[myfile, "Summarize this book in 3 key points"]
    )
    
    print("Summary:")
    print(response.text)
    
    # Check token usage for the large context
    print(f"\nToken usage: {response.usage_metadata.total_token_count}")
    
except Exception as e:
    print(f"Error uploading file: {e}")
    print("Make sure the file exists and is accessible")

File uploaded successfully: files/xw1pj961w8lf
Summary:
Here are three key points summarizing *The Adventures of Tom Sawyer*:

1.  **A Spirited Boyhood in Missouri:** The novel vividly portrays Tom Sawyer's imaginative and mischievous adventures growing up in a small town along the Mississippi River, featuring iconic scenes like the whitewashing of the fence and his escapades playing pirates and Indians with his friends Joe Harper and Huckleberry Finn.
2.  **Witnessing a Murder and its Moral Burden:** The central conflict arises when Tom and Huck accidentally witness a murder in the graveyard, leading to a pact of secrecy, intense fear of Injun Joe, and a significant moral dilemma for Tom that culminates in him bravely testifying at Muff Potter's trial.
3.  **The Quest for Treasure and Coming of Age:** Their ongoing pursuit of hidden treasure, particularly with Huck, leads them into dangerous situations, including getting lost in a cave with Becky Thatcher and a final, thrilling confro

In [19]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=211, candidates_tokens_details=None, prompt_token_count=102026, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=102026)], thoughts_token_count=831, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=None, total_token_count=103068, traffic_type=None)

## 9. !! Exercise: Chat with a "Book" !!

Create an interactive chat session where you can "talk" to the book "Alice in Wonderland". You'll set up the chat with a specific persona for the AI and use the book's text as context for the conversation.

Task: 
- Download the text of "Alice in Wonderland" (a helper code block is provided).
- Upload the book's text file (`alice_in_wonderland.txt`) to the Gemini API using `client.files.upload()`.
- Create a chat session using `client.chats.create()`:
- Send an initial message to the chat session using `chat.send_message()`:
- Send at least one follow-up question to the chat session (e.g., "Explain the various methods of speech delivery in more detail") and print its response.

In [20]:
import requests

# Download Alice in Wonderland
book_text_url = "https://www.gutenberg.org/files/11/11-0.txt"
try:
    response_book_req = requests.get(book_text_url)
    response_book_req.raise_for_status()  # Raise an exception for bad status codes
    
    with open("alice_in_wonderland.txt", "w", encoding="utf-8") as f:
        f.write(response_book_req.text)
    print("Book downloaded successfully!")
    
except requests.RequestException as e:
    print(f"Error downloading book: {e}")

Book downloaded successfully!


In [26]:
# TODO:
try:
    myfile = client.files.upload(file="alice_in_wonderland.txt")
    print(f"File uploaded successfully: {myfile.name}")

    chat_session = client.chats.create(model=MODEL_ID)

    user_message1 = "Can you briefly describe the book?"
    print(f"User: {user_message1}")
    response1 = chat_session.send_message(message=[myfile, user_message1])
    
    # # Generate content using the uploaded file as context
    # response = client.models.generate_content(
    #     model=MODEL_ID, 
    #     contents=[myfile, "Summarize this book in 3 key points"]
    # )
    
    print("Model:")
    print(response1.text)
    
    # Check token usage for the large context
    print(f"\nToken usage: {response1.usage_metadata.total_token_count}")
    
except Exception as e:
    print(f"Error uploading file: {e}")
    print("Make sure the file exists and is accessible")

File uploaded successfully: files/8l8rb4ygs3ax
User: Can you briefly describe the book?
Model:
"Alice's Adventures in Wonderland" tells the story of a young girl named Alice who, bored with her mundane surroundings, follows a White Rabbit down a rabbit-hole into a fantastical world called Wonderland. Throughout her journey, Alice experiences bewildering changes in size, struggles to understand the illogical rules and conversations of the peculiar creatures she meets (such as the White Rabbit, a philosophical Caterpillar, a grinning Cheshire Cat, the Mad Hatter, the March Hare, and the Dormouse), and frequently faces the tyrannical Queen of Hearts and her constant threats of execution. The book culminates in a chaotic court trial before Alice wakes up to realize her entire adventure was a vivid dream.

Token usage: 40182


In [27]:
# TODO:
user_message2 = "What about the court trial? Why is it chaotic?"
print(f"User: {user_message2}")
response2 = chat_session.send_message(
    message=user_message2
)

print("Model:")
print(response2.text)

User: Can you briefly describe the book?
Model:
The court trial in "Alice's Adventures in Wonderland" is chaotic due to its utter **absurdity, illogic, and the arbitrary nature of its proceedings**. Here's why:

1.  **Arbitrary Authority:** The King acts as the judge, but he's incompetent and easily swayed, constantly changing his mind or inventing rules on the spot (like "Rule Forty-two: All persons more than a mile high to leave the court"). The Queen of Hearts is even worse, frequently shouting "Off with his head!" or "Off with her head!" without any real cause or evidence, and famously demanding "Sentence first—verdict afterwards."

2.  **Incompetent Jury:** The twelve jurors (various animals and birds, including Bill the Lizard) are more concerned with writing down their names or making notes of absurdities (like Alice's "stupid things!") than with actual evidence. They are easily influenced and engage in nonsensical calculations (like adding up dates and converting them to shilli

## Recap & Next Steps

**What You've Learned:**
- Basic text generation with `client.models.generate_content()` for single prompts
- Token counting and cost estimation for better resource management
- Streaming responses with `generate_content_stream()` for improved user experience
- Multi-turn conversations using `client.chats.create()` and chat sessions
- System instructions for consistent model behavior and personality
- Generation configuration parameters for fine-tuning responses
- Long context handling and file uploads with the File API
- Error handling and best practices for production applications

**Key Takeaways:**
- Monitor token usage to control costs and stay within limits
- Use streaming for interactive applications and long responses
- Configure parameters based on your use case (factual vs creative content)
- Implement proper error handling for robust applications
- System instructions are powerful for setting behavior and tone

**Next Steps:** Continue with [Part 2: Multimodal Capabilities](https://github.com/philschmid/gemini-2.5-ai-engineering-workshop/blob/main/notebooks/02-multimodal-capabilities.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/philschmid/gemini-2.5-ai-engineering-workshop/blob/main/notebooks/02-multimodal-capabilities.ipynb)

**More Resources:**
- [Text Generation Guide](https://ai.google.dev/gemini-api/docs/text-generation)
- [Token Counting Guide](https://ai.google.dev/gemini-api/docs/tokens)
- [Long Context Documentation](https://ai.google.dev/gemini-api/docs/long-context)
- [File API Documentation](https://ai.google.dev/gemini-api/docs/files)